In [3]:
sc.stop()
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

conf = SparkConf().setAppName("202-f")\
                  .setMaster("spark://10.120.26.200:7077")\
                  .set("spark.executor.memory", '6g')\
                  .set('spark.cores.max', '4')

sc = SparkContext(conf=conf)
spark = SparkSession.builder.appName('sql coming~').getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

In [26]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from geoip2.database import Reader

def ip2city_py(ip):
    global reader
    if reader is None:
        # assume all work nodes have mmdb installed in the following path
        reader = Reader("/home/spark/spark-2.4.6-bin-hadoop2.7/maxmind/GeoLite2-City.mmdb")
    try:
        response = reader.city(ip)
        city = response.city.name
        if city is None:
            return None
        return city

    except:
        return None

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
ip2city= udf(ip2city_py, StringType())
ip2city

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<function ip2city_py at 0x7f142d05de60>

In [28]:
page_view = spark.read.csv("hdfs://10.120.26.200/user/spark/spark_sql_101/page_views/data",sep="\t",schema="logtime STRING, userid INT, ip STRING, page STRING, ref STRING, os STRING, os_ver STRING, agent STRING")
page_view.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------+---------------+---------------+----------+-------+------+-------+
|            logtime|userid|             ip|           page|       ref|     os|os_ver|  agent|
+-------------------+------+---------------+---------------+----------+-------+------+-------+
|08/05/2013 16:02:10|    21|  112.99.95.101|     DetailPage|Google.com|   Win8|     2| iPhone|
|08/05/2013 16:02:10|    34|  88.133.108.54|   ShoppingCart|Google.com|    Web|     4|    IE9|
|08/05/2013 16:02:10|    69|  126.147.94.49|     Appliances|Google.com|Android|     3|Samsung|
|08/05/2013 16:02:10|    10|   178.4.56.132|     SearchPage|Google.com|   Win8|     5| iPhone|
|08/05/2013 16:02:10|    55|  219.177.73.55|Recommendations|Google.com|   Win8|     3|    HTC|
|08/05/2013 16:02:10|    81|  71.67.178.110|     Appliances|Google.com|Android|     3| iPhone|
|08/05/2013 16:02:10|     9|  66.154.62.103|       HomePage|Google.com|    iOS|     3|    HTC|
|08/05/2013 16:02:10|    45| 121.158.57.158|     D

In [30]:
page_view_city = page_view.withColumn("city", ip2city("ip"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
stats_by_city_sorted = page_view_city.fillna("unknown", subset=["city"]) \
                                         .groupBy(col("city")) \
                                         .agg(count("*").alias("records"), countDistinct("userid").alias("UU")) \
                                         .orderBy(col("records").desc())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…